<a href="https://colab.research.google.com/github/rohitsharma21791/COVID_Directory/blob/main/efficientNetB0_exact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 23 21:46:01 2021

@author: Sharma
"""


from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from tensorflow.keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from tensorflow.keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU, add




from time import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
import math

from tensorflow.keras import backend as k

NUM_CLASSES = 1


def drop_connect(inputs, is_training, drop_connect_rate):
  """Apply drop connect.
  Args:
    inputs: `Tensor` input tensor.
    is_training: `bool` if True, the model is in training mode.
    drop_connect_rate: `float` drop connect rate.
  Returns:
    A output tensor, which should have the same shape as input.
  """
  if not is_training or drop_connect_rate is None or drop_connect_rate == 0:
    return inputs

  keep_prob = 1.0 - drop_connect_rate
  batch_size = tf.shape(inputs)[0]
  random_tensor = keep_prob
  random_tensor += tf.random.uniform([batch_size, 1, 1, 1], dtype=inputs.dtype)
  binary_tensor = tf.floor(random_tensor)
  output = tf.divide(inputs, keep_prob) * binary_tensor
  return output






def swish(x):
        return x * k.sigmoid(x)

def EfficientNet(n_classes,width_coefficient,depth_coefficient,resolution,dropout_rate,drop_connect_rate=0.2):
    input_shape=resolution,resolution,3
    
    


    def round_filters(filters, multiplier):
        depth_divisor =8
        min_depth = None
        min_depth = min_depth or depth_divisor
        filters = filters * multiplier
        new_filters = max(min_depth, int(filters + depth_divisor / 2) // depth_divisor * depth_divisor)
        if new_filters < 0.9 * filters:
            new_filters += depth_divisor
        return int(new_filters)



    def round_repeats(repeats, multiplier):
        if not multiplier:
            return repeats
        return int(math.ceil(multiplier * repeats))


    def SEBlock(x1,input_channels, ratio=0.25):
        num_reduced_filters = max(1, int(input_channels * ratio))
        x=GlobalAvgPool2D()(x1)
        x=tf.expand_dims(input=x,axis=1)
        x=tf.expand_dims(input=x,axis=1)
        x=Conv2D(filters=num_reduced_filters,kernel_size=(1,1),strides=1,padding="same")(x)
        x=swish(x)
        x=Conv2D(filters=input_channels,kernel_size=(1,1),strides=1,padding="same")(x)
        x=tf.nn.sigmoid(x)    #########################################################################################
        output=x1*x
        return output
    
    def SEBlock1(x1,input_channels, ratio=24):
        num_reduced_filters = max(1, int(input_channels // ratio))
        x=GlobalAvgPool2D()(x1)
        x=tf.expand_dims(input=x,axis=1)
        x=tf.expand_dims(input=x,axis=1)
        x=Conv2D(filters=num_reduced_filters,kernel_size=(1,1),strides=1,padding="same")(x)
        x=swish(x)
        x=Conv2D(filters=input_channels,kernel_size=(1,1),strides=1,padding="same")(x)
        x=tf.nn.sigmoid(x)    #########################################################################################
        output=x1*x
        return output
  
     

    def MBConv(x1,in_channels,out_channels,expansion_factor, stride, k, drop_connect_rate):
        
        
        x=DepthwiseConv2D(kernel_size=(k,k),strides=stride,padding="same",use_bias=False)(x1)
        x=BatchNormalization()(x)
        x=swish(x)
        x=SEBlock(x,input_channels=in_channels*expansion_factor)
        
        x=Conv2D(filters=out_channels,kernel_size=(1,1),strides=1,padding="same",use_bias=False)(x)
        x=BatchNormalization()(x)
        if stride==1 and in_channels== out_channels:
            if drop_connect_rate:
                x=drop_connect(x,is_training=bool,drop_connect_rate=0.2)
            x=add([x1,x])
        return x
    
    
    def MBConv1(x1,in_channels,out_channels,expansion_factor, stride, k, drop_connect_rate):
        
        x=Conv2D(filters=in_channels*expansion_factor,kernel_size=(1,1),strides=1,padding="same",use_bias=False)(x1)
        x=BatchNormalization()(x)
        x=swish(x)
        x=DepthwiseConv2D(kernel_size=(k,k),strides=stride,padding="same",use_bias=False)(x)
        x=BatchNormalization()(x)
        x=swish(x)
        x=SEBlock1(x,input_channels=in_channels*expansion_factor)
        
        x=Conv2D(filters=out_channels,kernel_size=(1,1),strides=1,padding="same",use_bias=False)(x)
        x=BatchNormalization()(x)
        if in_channels== out_channels:
            if drop_connect_rate:
                x=drop_connect(x,is_training=bool,drop_connect_rate=0.2)
                x=add([x1,x])
        return x

    def build_mbconv_block(x,in_channels, out_channels,layers, stride, expansion_factor, k, drop_connect_rate):
        for i in range(layers):
            if i==0:
                x=MBConv(x,in_channels=in_channels,out_channels=out_channels,expansion_factor=expansion_factor, stride=stride, k=k, drop_connect_rate=drop_connect_rate)
            else:
                x=MBConv(x,in_channels=out_channels,out_channels=out_channels,expansion_factor=expansion_factor, stride=1, k=k, drop_connect_rate=drop_connect_rate)
        return x
    
    
    def build_mbconv_block1(x,in_channels, out_channels,layers, stride, expansion_factor, k, drop_connect_rate):
        for i in range(layers):
            if i==0:
                x=MBConv1(x,in_channels=in_channels,out_channels=out_channels,expansion_factor=expansion_factor, stride=stride, k=k, drop_connect_rate=drop_connect_rate)
            else:
                x=MBConv1(x,in_channels=out_channels,out_channels=out_channels,expansion_factor=expansion_factor, stride=1, k=k, drop_connect_rate=drop_connect_rate)
        return x
            




    input=Input(input_shape)
    x=Conv2D(filters=round_filters(32,width_coefficient),kernel_size=(3,3),strides=2,padding="same",use_bias=False)(input)
    x=BatchNormalization()(x)
    x=swish(x)
    x= build_mbconv_block(x,in_channels=round_filters(32,width_coefficient),out_channels=round_filters(16,width_coefficient),layers=round_repeats(1, depth_coefficient),stride=1,expansion_factor=1,k=3, drop_connect_rate=drop_connect_rate)
    x= build_mbconv_block1(x,in_channels=round_filters(16,width_coefficient),out_channels=round_filters(24,width_coefficient),layers=round_repeats(2, depth_coefficient),stride=2,expansion_factor=6,k=3, drop_connect_rate=drop_connect_rate)
    x= build_mbconv_block1(x,in_channels=round_filters(24,width_coefficient),out_channels=round_filters(40,width_coefficient),layers=round_repeats(2, depth_coefficient),stride=2,expansion_factor=6,k=5, drop_connect_rate=drop_connect_rate)
    x= build_mbconv_block1(x,in_channels=round_filters(40,width_coefficient),out_channels=round_filters(80,width_coefficient),layers=round_repeats(3, depth_coefficient),stride=2,expansion_factor=6,k=3, drop_connect_rate=drop_connect_rate)
    x= build_mbconv_block1(x,in_channels=round_filters(80,width_coefficient),out_channels=round_filters(112,width_coefficient),layers=round_repeats(3, depth_coefficient),stride=1,expansion_factor=6,k=5, drop_connect_rate=drop_connect_rate)
    x= build_mbconv_block1(x,in_channels=round_filters(112,width_coefficient),out_channels=round_filters(192,width_coefficient),layers=round_repeats(4, depth_coefficient),stride=2,expansion_factor=6,k=5, drop_connect_rate=drop_connect_rate)          
    x= build_mbconv_block1(x,in_channels=round_filters(192,width_coefficient),out_channels=round_filters(320,width_coefficient),layers=round_repeats(1, depth_coefficient),stride=1,expansion_factor=6,k=3, drop_connect_rate=drop_connect_rate) 
    x=Conv2D(filters=round_filters(1280, width_coefficient),kernel_size=(1,1),strides=1,padding="same",use_bias=False)(x)      
    x=BatchNormalization()(x)
    x=swish(x)
    x=GlobalAvgPool2D()(x)
    x=Dropout(rate=dropout_rate)(x)
    output=Dense(units=NUM_CLASSES, activation=tf.keras.activations.sigmoid)(x)
    model = Model(input, output)
    return model



n_classes=1
batch_size=32
resolution=224
model=EfficientNet(n_classes,width_coefficient=1, depth_coefficient=1, resolution=224,dropout_rate=0.2)
model.summary()

          
            
            
            
    
    
    
    
    
    


    

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_520 (Conv2D)             (None, 112, 112, 32) 864         input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization_392 (BatchN (None, 112, 112, 32) 128         conv2d_520[0][0]                 
__________________________________________________________________________________________________
tf.math.sigmoid_520 (TFOpLambda (None, 112, 112, 32) 0           batch_normalization_392[0][0]    
____________________________________________________________________________________________